## 1. Nsight Systems Overview

### Nsight Systems vs Nsight Compute

| Feature | Nsight Systems | Nsight Compute |
|---------|---------------|----------------|
| **Scope** | Application-level | Kernel-level |
| **View** | Timeline | Metrics |
| **Use For** | CPU-GPU interaction | Kernel optimization |
| **Overhead** | Low | High |
| **Profiling** | All kernels at once | One kernel at a time |

### When to Use Nsight Systems

- Finding **where time is spent** in your application
- Identifying **CPU bottlenecks** between kernel launches
- Analyzing **memory transfer patterns**
- Detecting **synchronization issues**
- Understanding **stream and concurrency** behavior
- Profiling **multi-GPU** applications

### Installation Check

```bash
# Check if nsys is installed
nsys --version

# Typical output:
# NVIDIA Nsight Systems version 2024.1.1.59
```

### 🔷 CUDA C++ Implementation (Primary)

## 2. CUDA C++ Test Application

This application demonstrates various execution patterns that affect timeline behavior:
- **Pattern 1**: Synchronous execution (BAD - sync after every kernel)
- **Pattern 2**: Asynchronous execution (GOOD - single sync at end)
- **Pattern 3**: CPU-GPU overlap opportunity
- **Pattern 4**: Memory transfer patterns (pageable vs pinned)
- **Pattern 5**: Multiple CUDA streams for concurrency

In [ ]:
%%writefile app_profiling.cu
// app_profiling.cu - Application with various patterns to profile
#include <cuda_runtime.h>
#include <stdio.h>
#include <chrono>
#include <thread>

#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, \
               cudaGetErrorString(err)); \
        exit(1); \
    } \
}

//=============================================================================
// Kernels
//=============================================================================
__global__ void processKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float val = data[idx];
        for (int i = 0; i < 100; i++) {
            val = val * 1.01f + 0.01f;
        }
        data[idx] = val;
    }
}

__global__ void reduceKernel(float* output, const float* input, int n) {
    __shared__ float sdata[256];
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? input[idx] : 0.0f;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) sdata[tid] += sdata[tid + s];
        __syncthreads();
    }
    
    if (tid == 0) output[blockIdx.x] = sdata[0];
}

//=============================================================================
// Pattern 1: Synchronous execution (BAD - many sync points)
//=============================================================================
void patternSynchronous(float* d_data, int n, int iterations) {
    printf("Pattern 1: Synchronous (sync after every kernel)\n");
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    for (int i = 0; i < iterations; i++) {
        processKernel<<<gridSize, blockSize>>>(d_data, n);
        CHECK_CUDA(cudaDeviceSynchronize());  // Sync every time!
    }
}

//=============================================================================
// Pattern 2: Asynchronous execution (GOOD - batch sync)
//=============================================================================
void patternAsynchronous(float* d_data, int n, int iterations) {
    printf("Pattern 2: Asynchronous (sync at end)\n");
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    for (int i = 0; i < iterations; i++) {
        processKernel<<<gridSize, blockSize>>>(d_data, n);
        // No sync inside loop!
    }
    CHECK_CUDA(cudaDeviceSynchronize());  // Single sync at end
}

//=============================================================================
// Pattern 3: CPU work between kernels (potential overlap)
//=============================================================================
void cpuWork(int duration_us) {
    auto start = std::chrono::high_resolution_clock::now();
    while (true) {
        auto now = std::chrono::high_resolution_clock::now();
        auto elapsed = std::chrono::duration_cast<std::chrono::microseconds>(
            now - start).count();
        if (elapsed >= duration_us) break;
    }
}

void patternOverlap(float* d_data, int n, int iterations) {
    printf("Pattern 3: CPU-GPU overlap opportunity\n");
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    for (int i = 0; i < iterations; i++) {
        processKernel<<<gridSize, blockSize>>>(d_data, n);
        cpuWork(100);  // Simulated CPU work (could overlap)
    }
    CHECK_CUDA(cudaDeviceSynchronize());
}

//=============================================================================
// Pattern 4: Memory transfer patterns
//=============================================================================
void patternMemoryTransfers(int n) {
    printf("Pattern 4: Memory transfers\n");
    
    float *h_data, *d_data;
    size_t size = n * sizeof(float);
    
    // Pageable memory (slow)
    h_data = new float[n];
    CHECK_CUDA(cudaMalloc(&d_data, size));
    
    // Multiple small transfers (inefficient)
    printf("  Small transfers (pageable)...\n");
    for (int i = 0; i < 10; i++) {
        CHECK_CUDA(cudaMemcpy(d_data, h_data, size/10, 
                              cudaMemcpyHostToDevice));
        CHECK_CUDA(cudaMemcpy(h_data, d_data, size/10,
                              cudaMemcpyDeviceToHost));
    }
    
    delete[] h_data;
    cudaFree(d_data);
    
    // Pinned memory (fast)
    CHECK_CUDA(cudaMallocHost(&h_data, size));
    CHECK_CUDA(cudaMalloc(&d_data, size));
    
    // Single large transfer (efficient)
    printf("  Large transfer (pinned)...\n");
    CHECK_CUDA(cudaMemcpy(d_data, h_data, size, cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(h_data, d_data, size, cudaMemcpyDeviceToHost));
    
    cudaFreeHost(h_data);
    cudaFree(d_data);
}

//=============================================================================
// Pattern 5: Stream concurrency
//=============================================================================
void patternStreams(float* d_data, int n) {
    printf("Pattern 5: Multiple streams\n");
    
    const int NUM_STREAMS = 4;
    cudaStream_t streams[NUM_STREAMS];
    
    for (int i = 0; i < NUM_STREAMS; i++) {
        CHECK_CUDA(cudaStreamCreate(&streams[i]));
    }
    
    int blockSize = 256;
    int chunkSize = n / NUM_STREAMS;
    int gridSize = (chunkSize + blockSize - 1) / blockSize;
    
    // Launch work on multiple streams
    for (int i = 0; i < NUM_STREAMS; i++) {
        float* d_chunk = d_data + i * chunkSize;
        for (int j = 0; j < 5; j++) {
            processKernel<<<gridSize, blockSize, 0, streams[i]>>>(
                d_chunk, chunkSize);
        }
    }
    
    // Sync all streams
    for (int i = 0; i < NUM_STREAMS; i++) {
        CHECK_CUDA(cudaStreamSynchronize(streams[i]));
        CHECK_CUDA(cudaStreamDestroy(streams[i]));
    }
}

//=============================================================================
// Main
//=============================================================================
int main() {
    const int N = 1 << 20;  // 1M elements
    const int ITERATIONS = 20;
    
    float* d_data;
    CHECK_CUDA(cudaMalloc(&d_data, N * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_data, 0, N * sizeof(float)));
    
    printf("\n=== Application Profiling Demo ===\n\n");
    
    patternSynchronous(d_data, N, ITERATIONS);
    printf("\n");
    
    patternAsynchronous(d_data, N, ITERATIONS);
    printf("\n");
    
    patternOverlap(d_data, N, ITERATIONS);
    printf("\n");
    
    patternMemoryTransfers(N);
    printf("\n");
    
    patternStreams(d_data, N);
    printf("\n");
    
    printf("=== Done! Profile with nsys to see timeline ===\n");
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -lineinfo -o app_profiling app_profiling.cu
!./app_profiling

## 3. Basic Nsight Systems Usage

### Command-Line Profiling

```bash
# Basic profiling
nsys profile ./app_profiling

# Save report with custom name
nsys profile -o my_report ./app_profiling

# Profile with CUDA trace
nsys profile --trace=cuda ./app_profiling

# Include NVTX annotations
nsys profile --trace=cuda,nvtx ./app_profiling

# CPU sampling
nsys profile --sample=cpu ./app_profiling
```

### View Report

```bash
# Open report in GUI
nsys-ui my_report.nsys-rep

# Generate statistics summary
nsys stats my_report.nsys-rep

# Export to specific format
nsys export --type=sqlite my_report.nsys-rep
```

## 4. Understanding the Timeline

### Timeline Components

```
┌─────────────────────────────────────────────────────────────────┐
│ CPU Thread (main)                                               │
│ ████░░░████░░░████░░░████░░░████░░░████░░░████░░░████░░░        │
│     ↑       ↑       ↑                                           │
│   sync    sync    sync (expensive!)                             │
├─────────────────────────────────────────────────────────────────┤
│ CUDA API Calls                                                  │
│ ▓▓▓ ▓▓▓ ▓▓▓ ▓▓▓ ▓▓▓ ▓▓▓ ▓▓▓                                    │
│ Launch Launch Launch...                                         │
├─────────────────────────────────────────────────────────────────┤
│ GPU Compute (Stream 0)                                          │
│   █████   █████   █████   █████   █████   █████   █████         │
│   kernel  kernel  kernel  kernel  kernel  kernel  kernel        │
├─────────────────────────────────────────────────────────────────┤
│ GPU Memory                                                      │
│ ▒▒▒▒▒▒▒▒▒                                                       │
│ H2D transfer                                                    │
└─────────────────────────────────────────────────────────────────┘
```

### What to Look For

| Pattern | Indicates |
|---------|----------|
| Gaps in GPU timeline | CPU bottleneck or synchronization |
| Many small kernels | Kernel launch overhead |
| Large memory regions | Transfer-heavy code |
| Non-overlapping streams | Missed concurrency opportunity |
| Long API calls | Blocking operations |

### 🔷 CUDA C++ Implementation (Primary)

## 5. NVTX Annotations

NVTX (NVIDIA Tools Extension) allows you to add custom annotations to mark regions in your code. These annotations appear in Nsight Systems timeline for easier analysis.

In [ ]:
%%writefile nvtx_example.cu
// nvtx_example.cu - Add custom annotations
#include <cuda_runtime.h>
#include <nvtx3/nvToolsExt.h>
#include <stdio.h>

// Helper macro for NVTX ranges
#define NVTX_RANGE_PUSH(name) nvtxRangePushA(name)
#define NVTX_RANGE_POP() nvtxRangePop()

__global__ void computeKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        data[idx] = data[idx] * 2.0f;
    }
}

void initializeData(float* h_data, int n) {
    NVTX_RANGE_PUSH("Initialize");
    for (int i = 0; i < n; i++) {
        h_data[i] = (float)i;
    }
    NVTX_RANGE_POP();
}

void processData(float* d_data, int n) {
    NVTX_RANGE_PUSH("Process");
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    for (int i = 0; i < 10; i++) {
        char marker[32];
        snprintf(marker, sizeof(marker), "Iteration %d", i);
        NVTX_RANGE_PUSH(marker);
        
        computeKernel<<<gridSize, blockSize>>>(d_data, n);
        
        NVTX_RANGE_POP();
    }
    cudaDeviceSynchronize();
    
    NVTX_RANGE_POP();
}

int main() {
    const int N = 1 << 20;
    
    NVTX_RANGE_PUSH("Application");
    
    // Allocate
    NVTX_RANGE_PUSH("Allocation");
    float* h_data = new float[N];
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    NVTX_RANGE_POP();
    
    // Initialize
    initializeData(h_data, N);
    
    // Transfer
    NVTX_RANGE_PUSH("H2D Transfer");
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);
    NVTX_RANGE_POP();
    
    // Process
    processData(d_data, N);
    
    // Transfer back
    NVTX_RANGE_PUSH("D2H Transfer");
    cudaMemcpy(h_data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);
    NVTX_RANGE_POP();
    
    // Cleanup
    NVTX_RANGE_PUSH("Cleanup");
    delete[] h_data;
    cudaFree(d_data);
    NVTX_RANGE_POP();
    
    NVTX_RANGE_POP();  // Application
    
    printf("Done! Profile with: nsys profile --trace=cuda,nvtx ./nvtx_example\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -O3 -lnvToolsExt -o nvtx_example nvtx_example.cu
!./nvtx_example

## 6. Common Issues and Solutions

### Issue 1: GPU Idle Time (Gaps in Timeline)

**Symptoms in timeline:**
```
GPU: ████    ████    ████    ████
         ^gap   ^gap   ^gap
```

**Causes:**
- Excessive `cudaDeviceSynchronize()` calls
- CPU work blocking kernel launch
- Serial kernel dependencies

**Solution:**
```cpp
// Bad: sync after each kernel
for (int i = 0; i < N; i++) {
    kernel<<<grid, block>>>(...);
    cudaDeviceSynchronize();  // Creates gap!
}

// Good: single sync at end
for (int i = 0; i < N; i++) {
    kernel<<<grid, block>>>(...);
}
cudaDeviceSynchronize();  // One sync
```

### Issue 2: Memory Transfer Overhead

**Symptoms:**
- Large memory transfer regions
- Many small transfers

**Solution:**
```cpp
// Bad: many small transfers
for (int i = 0; i < 100; i++) {
    cudaMemcpy(d_data + i*chunk, h_data + i*chunk, 
               chunk_size, cudaMemcpyHostToDevice);
}

// Good: single large transfer
cudaMemcpy(d_data, h_data, total_size, cudaMemcpyHostToDevice);
```

### Issue 3: No Stream Overlap

**Symptoms:**
- Multiple streams but sequential execution

**Common cause:** Implicit synchronization

```cpp
// These cause implicit sync:
cudaMalloc(...)           // Syncs
cudaMemcpy(...)           // Syncs (use Async version)
cudaDeviceSynchronize()   // Explicit sync

// Use async versions for overlap:
cudaMemcpyAsync(d_data, h_data, size, cudaMemcpyHostToDevice, stream);
```

## 7. Statistics Analysis

### Generate Statistics

```bash
# Profile and get stats
nsys profile -o report ./app_profiling
nsys stats report.nsys-rep
```

### Key Statistics Tables

```
CUDA API Statistics:
┌─────────────────────┬─────────┬──────────┬──────────┐
│ API                 │ Count   │ Avg (ns) │ Total %  │
├─────────────────────┼─────────┼──────────┼──────────┤
│ cudaDeviceSynchronize│ 100    │ 50000    │ 45.2%    │  ← Too many syncs!
│ cudaLaunchKernel    │ 100    │ 5000     │ 4.5%     │
│ cudaMemcpy          │ 20     │ 100000   │ 18.1%    │
└─────────────────────┴─────────┴──────────┴──────────┘

GPU Kernel Statistics:
┌────────────────────┬─────────┬──────────┬──────────┐
│ Kernel             │ Count   │ Avg (us) │ Total %  │
├────────────────────┼─────────┼──────────┼──────────┤
│ processKernel      │ 80     │ 125      │ 85.3%    │
│ reduceKernel       │ 20     │ 50       │ 14.7%    │
└────────────────────┴─────────┴──────────┴──────────┘

Memory Transfer Statistics:
┌────────────────────┬─────────┬──────────┬──────────┐
│ Transfer Type      │ Count   │ Size     │ BW (GB/s)│
├────────────────────┼─────────┼──────────┼──────────┤
│ H2D (Pageable)     │ 10     │ 1 MB     │ 5.2      │  ← Slow!
│ H2D (Pinned)       │ 1      │ 10 MB    │ 12.1     │  ← Fast
└────────────────────┴─────────┴──────────┴──────────┘
```

### 🔶 Python/Numba (Optional - Quick Testing)

## 8. Python/Numba Optional Backup

Simulate timeline concepts in Python:

In [ ]:
!pip install numba numpy matplotlib -q

In [ ]:
import numpy as np
import time
from numba import cuda

@cuda.jit
def process_kernel(data):
    idx = cuda.grid(1)
    if idx < data.size:
        val = data[idx]
        for i in range(100):
            val = val * 1.01 + 0.01
        data[idx] = val

# Check GPU
if not cuda.is_available():
    print("No CUDA GPU available")
else:
    print(f"GPU: {cuda.get_current_device().name}")

In [ ]:
# Compare synchronous vs asynchronous patterns

if cuda.is_available():
    N = 1 << 20  # 1M elements
    ITERATIONS = 20
    
    d_data = cuda.device_array(N, dtype=np.float32)
    
    threads = 256
    blocks = (N + threads - 1) // threads
    
    # Warm up
    process_kernel[blocks, threads](d_data)
    cuda.synchronize()
    
    # Pattern 1: Sync after every kernel (BAD)
    start = time.perf_counter()
    for i in range(ITERATIONS):
        process_kernel[blocks, threads](d_data)
        cuda.synchronize()  # Sync every time!
    sync_time = time.perf_counter() - start
    
    # Pattern 2: Single sync at end (GOOD)
    start = time.perf_counter()
    for i in range(ITERATIONS):
        process_kernel[blocks, threads](d_data)
    cuda.synchronize()  # Single sync
    async_time = time.perf_counter() - start
    
    print(f"Synchronous pattern:  {sync_time*1000:.2f} ms")
    print(f"Asynchronous pattern: {async_time*1000:.2f} ms")
    print(f"Speedup: {sync_time/async_time:.2f}x")
    print(f"\n→ Removing unnecessary syncs can significantly improve performance!")

In [ ]:
# Visualize timeline-like execution pattern
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def visualize_timeline(pattern_name, cpu_events, gpu_events):
    """Visualize CPU and GPU timeline"""
    fig, ax = plt.subplots(figsize=(14, 4))
    
    # CPU timeline
    for start, end, label, color in cpu_events:
        ax.barh(1, end-start, left=start, height=0.4, color=color, 
                edgecolor='black', linewidth=0.5)
        ax.text((start+end)/2, 1, label, ha='center', va='center', fontsize=8)
    
    # GPU timeline
    for start, end, label, color in gpu_events:
        ax.barh(0, end-start, left=start, height=0.4, color=color,
                edgecolor='black', linewidth=0.5)
    
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['GPU', 'CPU'])
    ax.set_xlabel('Time (arbitrary units)')
    ax.set_title(f'Timeline: {pattern_name}')
    ax.set_xlim(-0.5, max(e[1] for e in cpu_events + gpu_events) + 0.5)
    
    plt.tight_layout()
    plt.show()

# Pattern 1: Synchronous (gaps in GPU)
cpu_sync = [(0, 0.5, 'L', 'lightblue'), (0.5, 1.0, 'S', 'red'),
            (1.5, 2.0, 'L', 'lightblue'), (2.0, 2.5, 'S', 'red'),
            (3.0, 3.5, 'L', 'lightblue'), (3.5, 4.0, 'S', 'red'),
            (4.5, 5.0, 'L', 'lightblue'), (5.0, 5.5, 'S', 'red')]
gpu_sync = [(0.5, 1.5, 'K', 'green'), (2.0, 3.0, 'K', 'green'),
            (3.5, 4.5, 'K', 'green'), (5.0, 6.0, 'K', 'green')]

visualize_timeline('Synchronous (sync after each kernel)', cpu_sync, gpu_sync)

# Pattern 2: Asynchronous (continuous GPU)
cpu_async = [(0, 0.3, 'L', 'lightblue'), (0.3, 0.6, 'L', 'lightblue'),
             (0.6, 0.9, 'L', 'lightblue'), (0.9, 1.2, 'L', 'lightblue'),
             (4.0, 4.5, 'S', 'red')]
gpu_async = [(0.3, 1.3, 'K', 'green'), (1.3, 2.3, 'K', 'green'),
             (2.3, 3.3, 'K', 'green'), (3.3, 4.3, 'K', 'green')]

visualize_timeline('Asynchronous (single sync at end)', cpu_async, gpu_async)

print("Legend: L=Launch, S=Sync, K=Kernel")
print("\nNotice the gaps in Pattern 1 vs continuous execution in Pattern 2")

## 9. Best Practices for Application Profiling

### Profiling Workflow

1. **Start with Nsight Systems** to understand application-level behavior
2. **Look for gaps** in GPU timeline (idle time)
3. **Check API call durations** for blocking operations
4. **Identify hot kernels** for detailed analysis
5. **Use Nsight Compute** on specific kernels for optimization

### Checklist

- [ ] GPU timeline is mostly busy (minimal gaps)
- [ ] Minimize `cudaDeviceSynchronize()` calls
- [ ] Use pinned memory for transfers
- [ ] Batch small transfers into larger ones
- [ ] Use streams for overlapping operations
- [ ] CPU work overlaps with GPU execution

### Command Quick Reference

```bash
# Basic profile
nsys profile ./app

# Full tracing
nsys profile --trace=cuda,nvtx,osrt ./app

# With CPU sampling
nsys profile --sample=cpu --trace=cuda ./app

# Statistics only
nsys stats report.nsys-rep
```

## 10. Exercises

### Exercise 1: Profile Sync Points
Profile the `app_profiling` program and identify how many sync points exist.

### Exercise 2: Add NVTX Markers
Add NVTX annotations to your own CUDA application to mark:
- Initialization phase
- Main compute loop
- Cleanup phase

### Exercise 3: Identify Bottleneck
Profile an application and determine if it's:
- GPU-bound (GPU busy, CPU waiting)
- CPU-bound (GPU idle, CPU busy)
- Transfer-bound (large memory transfer regions)

### Exercise 4: Optimize Sync Points
Take a program with excessive syncs and optimize by batching operations.

## Summary

Today you learned:
- **Nsight Systems** profiles application-level behavior
- **Timeline view** shows CPU-GPU interaction
- **Gaps in GPU timeline** indicate synchronization or CPU bottlenecks
- **NVTX annotations** add custom markers for clarity
- **Statistics** reveal API call patterns and kernel durations

### Key Commands
```bash
nsys profile ./app           # Profile
nsys stats report.nsys-rep   # Statistics
nsys-ui report.nsys-rep      # GUI view
```

**Next**: Day 4 - Systematic bottleneck analysis combining all profiling techniques